## Convolutional Neural Network
   #### Computer Vision Winter Semester 2020/2021 by Clemens Spielvogel
      
In the following code example a classification model for the MNIST data is built. Only few training data is used to demonstrate the increased performance provided by data augmentation in the next jupyter notebook (13).

In [1]:
import os
#os.environ["CUDA_VISIBLE_DEVICES"] = "" # If you want to use TF CPU version while having the GPU version installed

import tensorflow as tf
from keras import Sequential, layers, regularizers
import keras
import numpy as np
import pandas as pd
import plotly.offline as ply
import plotly.graph_objs as graphs
import math
import random
import matplotlib.pyplot as plt

Using TensorFlow backend.


In [2]:
# Seeding random number generators to obtain reproducible results
seed_value = 0
os.environ['PYTHONHASHSEED']=str(seed_value)
random.seed(seed_value)
np.random.seed(seed_value) # Resets itself on every use!
tf.random.set_seed(seed_value) # tf.set_random_seed(seed_value) on older TF versions

In [3]:
def load_data(file_path):
    # Load CSV file to pandas data frame
    df = pd.read_csv(file_path, header=0)

    # Split data frame columns into features and labels
    y = df["label"].values
    X_flat = df.drop("label", axis=1).values
    X = [np.resize(array, (28, 28)) for array in X_flat] # Resize flat vector to 28x28
    X = np.array([np.reshape(sample, (sample.shape[0], sample.shape[1], 1)) for sample in X]) # Add color channel
    
    return X, keras.utils.to_categorical(y) # Labels are returned as one-hot encoded vectors

In [4]:
def plot_train_val_performances(performances, metric="Accuracy", show=True):
    """Plot training vs. testing accuracy over all epochs. performances is a dictionary mapping epoch numbers
       as integers to lists containing training and validation performance (e.g. accuracy).
       metric is a string indicating the used performance metric."""
    x = list(performances.keys())     # Number of epoch
    y_train = [i[0] for i in performances.values()]
    y_val = [i[1] for i in performances.values()]

    trace_train = graphs.Scatter(x=x, y=y_train, name="Training", mode="lines+markers",
                                 line=dict(width=4),
                                 marker=dict(symbol="circle",
                                             size=10))
    trace_val = graphs.Scatter(x=x, y=y_val, name="Validation", mode="lines+markers",
                                line=dict(width=4),
                                marker=dict(symbol="circle",
                                            size=10))

    layout = graphs.Layout(title="Training vs. Validation {}".format(metric),
                           xaxis={"title": "Epoch"},
                           yaxis={"title": metric})

    fig = graphs.Figure(data=[trace_train, trace_val], layout=layout)
    ply.plot(fig, filename="plotly_train_val_{}.html".format(metric), auto_open=show)
    print("Plot saved as plotly_train_val_{}.html".format(metric))

In [5]:
def create_2DCNN_model(input_shape):
    """Build architecture of the model"""
    model = Sequential()
    model.add(layers.Conv2D(32, (3, 3), input_shape=input_shape,
                            activation="relu", padding="same"))
    model.add(layers.Conv2D(64, (3, 3), activation="selu", padding="same"))
    model.add(layers.MaxPooling2D(pool_size=(3, 3)))
    model.add(layers.Conv2D(64, (3, 3), activation="selu", padding="same"))
    model.add(layers.Conv2D(64, (3, 3), activation="selu", padding="same"))
    model.add(layers.MaxPooling2D(pool_size=(2, 2)))
    model.add(layers.Conv2D(128, (3, 3), activation="selu", padding="same"))
    model.add(layers.MaxPooling2D(pool_size=(2, 2), padding="same"))
    model.add(layers.Flatten())
    model.add(layers.Dense(64, activation="selu",
                           kernel_regularizer=regularizers.l2(0.001)))
    model.add(layers.Dropout(0.2))
    model.add(layers.Dense(32, activation="selu"))
    model.add(layers.Dense(10, activation="softmax"))

    # Create model
    model.compile(optimizer=tf.keras.optimizers.Adam(),
                  loss="categorical_crossentropy",
                  metrics=["accuracy"])

    return model

In [6]:
# Specify data location
data_dir = r"Data\mnist.csv"

# Set hyperparameters
num_epochs = 50
batch_size = 64
dims = (28, 28, 1)

In [7]:
# Load data
X, y = load_data(data_dir)

# Determine split sizes (Could also be hardcoded)
train_size = math.floor(0.02 * len(y))  # ONLY 2 % of the data set is used for training to demonstrate the augmentation benefit
val_size = math.floor(0.49 * len(y))
test_size = math.floor(0.49 * len(y))

# Create splitted sets
X_train, y_train = X[:train_size], y[:train_size]
X_val, y_val = X[train_size:train_size+val_size], y[train_size:train_size+val_size]
X_test, y_test = X[train_size+val_size:], y[train_size+val_size:]

In [8]:
# Create CNN model
model = create_2DCNN_model(dims)

In [9]:
# Configure stopping criterion via early stopping
callback = tf.keras.callbacks.EarlyStopping(monitor="loss", patience=4)

In [10]:
# Train model
train_summary = model.fit(x=X_train,
                          y=y_train,
                          validation_data=(X_val, y_val),
                          batch_size=batch_size,
                          epochs=num_epochs,
                          callbacks=[callback],
                          verbose=1)

Train on 200 samples, validate on 4900 samples
Epoch 1/50
200/200 [==============================] - 2s 12ms/step - loss: 10.9522 - accuracy: 0.0850 - val_loss: 3.2980 - val_accuracy: 0.1633
Epoch 2/50
200/200 [==============================] - 1s 3ms/step - loss: 4.1512 - accuracy: 0.1500 - val_loss: 2.8146 - val_accuracy: 0.2027
Epoch 3/50
200/200 [==============================] - 1s 3ms/step - loss: 3.2469 - accuracy: 0.2250 - val_loss: 2.1239 - val_accuracy: 0.3694
Epoch 4/50
200/200 [==============================] - 1s 3ms/step - loss: 2.2322 - accuracy: 0.3400 - val_loss: 2.0041 - val_accuracy: 0.4073
Epoch 5/50
200/200 [==============================] - 1s 3ms/step - loss: 2.0541 - accuracy: 0.4450 - val_loss: 1.8749 - val_accuracy: 0.4420
Epoch 6/50
200/200 [==============================] - 1s 3ms/step - loss: 1.6047 - accuracy: 0.5300 - val_loss: 1.5729 - val_accuracy: 0.5096
Epoch 7/50
200/200 [==============================] - 1s 3ms/step - loss: 1.5218 - accuracy: 0.5250

In [11]:
# Evaluate fitted model using test data
test_loss, test_acc = model.evaluate(X_test, y_test, verbose=1)
print("\nTest ACC:", round(test_acc, 3))

4900/4900 [==============================] - 1s 186us/step

Test ACC: 0.833


In [12]:
# Get epochwise performances
train_acc = train_summary.history["accuracy"]
val_acc = train_summary.history["val_accuracy"]

# Format and store performances per epoch for plotting
accs = {epoch: [round(performance[0], 2), round(performance[1], 2)]
        for epoch, performance in enumerate(zip(train_acc, val_acc))}

# Plot training and validation performance over epochs
plot_train_val_performances(accs, "Accuracy")

Plot saved as plotly_train_val_Accuracy.html


In [13]:
# Save model
model.save_weights("model_weights.h5")